In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports for utilities
import os
from utils.config import Config  # Pydantic data model for config
from utils.logging import init_logging
from utils.seed import set_seed

# Define configuration as a nested dictionary (or use JSON format)
config_dict = {
    "system": {
        "numexpr_threads": 4,
        "deterministic": False,
        "cpu": {
          "intra_op_threads": None,
          "inter_op_threads": None,
          "omp_num_threads": None,
          "mkl_num_threads": None,
          "kmp_affinity": "granularity=fine,compact,1,0",
          "start_method": "forkserver",
          "pin_affinity_cores": None
        },
        "dataloader": {
          "num_workers": os.cpu_count()//2,           # auto -> max(2, cores-1)
          "prefetch_factor": 4,
          "persistent_workers": True,
          "pin_memory": True,
          "pin_memory_device": "cuda",
          "non_blocking": True
        },
        "cuda": {
          "enabled": True,
          "device_ids": None,            # auto -> all visible
          "allow_tf32": True,
          "matmul_precision": "high",
          "cudnn_benchmark": True,
          "cudnn_deterministic": None,
          "amp": True,
          "amp_dtype": "bf16",           # prefer bf16 on Ampere+ if available
          "grad_scaler_enabled": True,
          "compile_mode": "reduce-overhead",
          "compile_fullgraph": False,
          "uva": False
        },
        "ddp": {
          "enabled": False,
          "backend": "nccl",
          "find_unused_parameters": False,
          "gradient_as_bucket_view": True,
          "broadcast_buffers": False,
          "static_graph": False
        }
    },
    "data": {
        "csv_path": "raw/diabetic_data.csv",            # raw data file
        "ids_mapping_path": "raw/IDS_mapping.csv",          # ID mapping file for codes
        "target": {"name": "readmitted", "positive_values": ["<30"], "binarized_name": "readmitted_binary"},  # predict 30-day readmit
        "identifier_cols": {"encounter_id": "encounter_id", "patient_id": "patient_nbr"},
        "filters": {
            "exclude_discharge_to_ids": [11, 13, 14, 19, 20, 21],
            "first_encounter_per_patient": True,
        },
        "columns": {
            # Numeric features (counts, etc.)
            "numeric": ["time_in_hospital", "num_lab_procedures", "num_procedures",
                        "num_medications", "number_outpatient", "number_emergency",
                        "number_inpatient", "number_diagnoses"],
            # Low-cardinality categoricals (will be one-hot or label encoded as features)
            "categorical_low_card": ["race", "gender", "age", "max_glu_serum", "A1Cresult", "change", "diabetesMed"],
            # High-cardinality categorical columns to be turned into separate nodes
            "icd_cols": ["diag_1", "diag_2", "diag_3"],          # diagnosis code columns
            "drug_cols": ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride",
                          "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone",
                          "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide",
                          "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin",
                          "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"],
            "hospital_col": None,                                # (dataset has no explicit hospital ID column)
            "specialty_col": "medical_specialty",                # physician specialty
            "admission_type_col": "admission_type_id",
            "discharge_disposition_col": "discharge_disposition_id",
            "admission_source_col": "admission_source_id"
        },
        "preprocessing": {
            "numeric_imputer": "mean",           # impute missing numeric with mean
            "categorical_imputer": "most_frequent",  # impute missing categoricals with mode
            "unknown_label": "UNKNOWN",          # label for unseen or rare categories
            "use_unknown_category": True,        # add an "UNKNOWN" category for unseen values
            "min_freq_for_category": 5,          # rare category threshold (below this -> UNKNOWN)
            "truncate_icd_to_3_digits": True     # use only first 3 digits of ICD codes to group
        },
        "splits": {
            "group_by": "patient",    # group splits by patient_id to avoid leakage:contentReference[oaicite:6]{index=6}
            "n_splits": 5,           # use 5-fold split (first fold for train/val, second for test)
            "stratify_by_target": True,
            "seed": 42
        }
    },
    "graph": {
        # Enable various node and edge types in the heterogeneous graph
        "node_types_enabled": {
            "encounter": True, "icd": True, "icd_group": True, "drug": True, "drug_class": True,
            "specialty": True, "admission_type": True, "discharge_disposition": True, "admission_source": True,
            "hosp": False  # no hospital node since not in dataset
        },
        "edge_types_enabled": {
            "encounter__has_icd__icd": True,
            "icd__is_a__icd_group": True,
            "encounter__has_drug__drug": True,
            "drug__belongs_to__drug_class": True,
            "encounter__has_specialty__specialty": True,
            "encounter__has_admission_type__admission_type": True,
            "encounter__has_discharge__discharge_disposition": True,
            "encounter__has_admission_source__admission_source": True,
            "reverse_edges": True    # add reverse of every relation for undirected information flow
        },
        "edge_featuring": {
            "has_drug": {
                "relation_subtypes_by_status": True,  # separate edge types for Up/Down/Steady drug status
                "edge_attr_status": True              # include an edge attribute indicating drug change
            }
        },
        "fanouts": {
            # Neighbor sampling fanout per edge type per GNN layer (2-layer example):
            "encounter__has_icd__icd": [10, 5],
            "encounter__has_drug__drug": [5, 5],
            "encounter__has_specialty__specialty": [-1],  # -1 means take all neighbors (specialty has 1 neighbor per encounter)
            "encounter__has_admission_type__admission_type": [-1],
            "encounter__has_discharge__discharge_disposition": [-1],
            "encounter__has_admission_source__admission_source": [-1],
            "icd__is_a__icd_group": [-1],
            "drug__belongs_to__drug_class": [-1],
            "reverse_edges": [5, 5]  # sample some reverse edges if needed
        }
    },
    "model": {
        "arch": "HGT",           # model architecture: "HGT", "RGCN", or "GraphSAGE"
        "hidden_dim": 64,        # hidden embedding size
        "num_layers": 2,         # number of GNN layers
        "heads": 2,              # number of attention heads (for HGT)
        "rgcn_bases": 0       # number of bases for RGCN
    },
    "train": {
        "epochs": 30,
        "early_stopping_patience": 5,
        "val_every": 1,          # evaluate on val every epoch
        "gradient_clip_norm": 2.0,
        "optimizer": {
            "name": "Adam",
            "lr":  0.001,
            "weight_decay": 0.05,
        },
        "batching": {
            "batch_size_encounters": 128
        }
    },
    "inference": {
        "output_predictions_path": "artifacts/predictions.csv"
    },
    "evaluation":{
        "metrics_primary": [],
        "metrics_secondary": [],
    },
    "baseline": {},
    "path": {
        "artifacts_dir": "artifacts/",
        "tb_log_dir": "artifacts/tb_logs/",
        "logging_path": "logs/"
    }
}

# Initialize config object
config = Config(**config_dict)

In [3]:
from utils.system import apply_system_config  # or from the cell above
rt = apply_system_config(config)
device = rt["device"]

# Set random seeds for reproducibility
set_seed(config, 42)

# Initialize logging and TensorBoard writer
logger, writer = init_logging(config.path.logs_dir)
logger.info("Configuration and logging initialized.")

2025-10-22 16:38:26 INFO:Configuration and logging initialized.


numexpr threads: 4


In [4]:
import pandas as pd
from data import loader

# Load the datasets
df = loader.load_csv(config.data.csv_path)
df = loader.apply_filters(df, config)

logger.info(f"Raw data shape: {df.shape}")
logger.info(f"Columns: {list(df.columns)}")

2025-10-22 16:38:28 INFO:Raw data shape: (69990, 50)
2025-10-22 16:38:28 INFO:Columns: ['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted']


In [5]:
from data import preprocess

# Create train/val/test splits first (to fit imputer/scaler on train only)
from data.splits import create_splits
splits = create_splits(df, config)
train_idx, val_idx, test_idx = splits

df_train = df.iloc[train_idx].copy().reset_index(drop=True)
df_val = df.iloc[val_idx].copy().reset_index(drop=True)
df_test = df.iloc[test_idx].copy().reset_index(drop=True) if test_idx is not None else None

logger.info(f"Split sizes -> Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

# Preprocess the splits
df_train, df_val, df_test, scaler = preprocess.preprocess_data(df_train, df_val, df_test, config)

# The scaler and any encodings from train are now ready for use in inference too
logger.info("Preprocessing complete. Sample of processed features:")
logger.info(df_train[config.data.columns.numeric + config.data.columns.categorical_low_card].head(3))


2025-10-22 16:38:29 INFO:Split sizes -> Train: 55992, Val: 13998, Test: 13998
2025-10-22 16:38:30 INFO:Preprocessing complete. Sample of processed features:
2025-10-22 16:38:30 INFO:   time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0          2.970598            1.261342        0.329610         1.482870   
1          2.629817           -0.497552        0.899788         0.278178   
2          1.607474            0.206006        0.329610         0.157708   

   number_outpatient  number_emergency  number_inpatient  number_diagnoses  \
0          -0.260879         -0.198924          -0.29252          0.386081   
1          -0.260879         -0.198924          -0.29252          0.386081   
2          -0.260879         -0.198924          -0.29252          0.886218   

              race  gender       age max_glu_serum A1Cresult change  \
0        Caucasian  Female   [80-90)          Norm        >8     Ch   
1        Caucasian  Female  [90-100)          Norm        

In [6]:
from data import vocab

vocabs, mappings = vocab.make_vocabs(df_train, config)
logger.info("Vocab sizes: " + ", ".join(f"{k}: {len(v)}" for k,v in vocabs.items()))


2025-10-22 16:38:30 INFO:Vocab sizes: icd: 844, icd_group: 844, drug: 23, drug_class: 1, admission_type: 9, discharge_disposition: 22, admission_source: 18, specialty: 68


In [7]:
# Verify no patient overlap between train, val, test
train_patients = set(df_train[config.data.identifier_cols.patient_id])
val_patients = set(df_val[config.data.identifier_cols.patient_id])
test_patients = set(df_test[config.data.identifier_cols.patient_id])

overlap_train_val = train_patients.intersection(val_patients)
overlap_train_test = train_patients.intersection(test_patients)
overlap_val_test = val_patients.intersection(test_patients)
logger.info(f"Patient overlap - Train/Val: {len(overlap_train_val)}, Train/Test: {len(overlap_train_test)}, Val/Test: {len(overlap_val_test)}")

# Ensure target stratification roughly preserved
mean_train = df_train[config.data.target.binarized_name].mean(); mean_val = df_val[config.data.target.binarized_name].mean(); mean_test = df_test[config.data.target.binarized_name].mean()
logger.info(f"Readmit rate - Train: {mean_train:.3f}, Val: {mean_val:.3f}, Test: {mean_test:.3f}")

2025-10-22 16:38:31 INFO:Patient overlap - Train/Val: 0, Train/Test: 13998, Val/Test: 0
2025-10-22 16:38:31 INFO:Readmit rate - Train: 0.089, Val: 0.091, Test: 0.090


In [8]:
from graph import builder

# Build heterogeneous graphs for each split
graph_train = builder.build_heterodata(df_train, vocabs, config, include_target=True)
graph_val   = builder.build_heterodata(df_val, vocabs, config, include_target=True)
graph_test  = builder.build_heterodata(df_test, vocabs, config, include_target=True)

# Log graph statistics
logger.info(f"Graph (Train) node types: {list(graph_train.node_types)}")
for ntype in graph_train.node_types:
    logger.info(f"  {ntype}: {graph_train[ntype].num_nodes} nodes")
logger.info(f"Graph (Train) edge types: {list(graph_train.edge_types)}")
for etype in graph_train.edge_types:
    logger.info(f"  {etype}: {graph_train[etype].edge_index.size(1)} edges")


2025-10-22 16:38:45 INFO:Graph (Train) node types: ['encounter', 'icd', 'icd_group', 'drug', 'drug_class', 'admission_type', 'discharge_disposition', 'admission_source', 'specialty']
2025-10-22 16:38:45 INFO:  encounter: 55992 nodes
2025-10-22 16:38:45 INFO:  icd: 844 nodes
2025-10-22 16:38:45 INFO:  icd_group: 844 nodes
2025-10-22 16:38:45 INFO:  drug: 23 nodes
2025-10-22 16:38:45 INFO:  drug_class: 1 nodes
2025-10-22 16:38:45 INFO:  admission_type: 9 nodes
2025-10-22 16:38:45 INFO:  discharge_disposition: 22 nodes
2025-10-22 16:38:45 INFO:  admission_source: 18 nodes
2025-10-22 16:38:45 INFO:  specialty: 68 nodes
2025-10-22 16:38:45 INFO:Graph (Train) edge types: [('encounter', 'has_icd', 'icd'), ('icd', 'rev_has_icd', 'encounter'), ('icd', 'is_a', 'icd_group'), ('icd_group', 'rev_is_a', 'icd'), ('encounter', 'has_drug_steady', 'drug'), ('drug', 'rev_has_drug_steady', 'encounter'), ('drug', 'belongs_to', 'drug_class'), ('drug_class', 'rev_belongs_to', 'drug'), ('encounter', 'has_spec

In [9]:
# Jupyter cell (with tqdm bars)
import torch
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import average_precision_score
from tqdm.auto import tqdm, trange  # pretty progress bars in notebooks/terminals

from train.model_factory import get_model_class, setup_and_compile_model
from train.optim import make_optimizer, make_scheduler
from train.sampling import build_num_neighbors
from train.losses import make_criterion
from train.device import get_device
# sizes from the TRAIN graph
enc_input_dim = graph_train['encounter'].x.size(-1)
type_vocab_sizes = {nt: graph_train[nt].num_nodes for nt in graph_train.node_types if nt != 'encounter'}

# optional one-time sanity check
def validate_indices(g):
    for nt in g.node_types:
        if nt == 'encounter':
            continue
        x = g[nt].x
        if x.numel() == 0:
            continue
        vmax = int(x.max().item())
        n = int(g[nt].num_nodes)
        assert vmax < n, f"[{nt}] max index {vmax} >= num_nodes {n}"
validate_indices(graph_train)

# build model (move to device after full construction)
device = get_device()
ModelClass = get_model_class(config)
metadata = (list(graph_train.node_types), list(graph_train.edge_types))
model = ModelClass(
    metadata,
    config,
    enc_input_dim=enc_input_dim,
    type_vocab_sizes=type_vocab_sizes,
).to(device)  # or pass device=device in the constructor instead
model = setup_and_compile_model(model, config, logger)

2025-10-22 16:38:46 INFO:Skipping torch.compile (Windows not supported). Set system.cuda.compile_mode=None to silence this permanently.


In [ ]:
from train.loop import train_one_epoch
from train.loader import make_neighbor_loader
# ---------------------------
# Optimizer, scheduler, loss
# ---------------------------
optimizer = make_optimizer(model.parameters(), config)
scheduler = make_scheduler(optimizer, config)
criterion = make_criterion(graph_train, config, device)

# ---------------------------
# NeighborLoader
# ---------------------------
num_layers = int(getattr(config.model, "num_layers", 2))
num_neighbors = build_num_neighbors(graph_train, config, num_layers)

batch_size = getattr(getattr(config.train, "batching", object()), "batch_size_encounters",
                     getattr(config.train, "batch_size", 1024))
train_loader = make_neighbor_loader(
    graph_train,
    input_nodes=("encounter", torch.arange(graph_train["encounter"].num_nodes)),
    num_neighbors=num_neighbors,
    config=config,
    train=True,
    shuffle=True
)

val_data = graph_val.to(device)

# ---------------------------
# Train loop (early stopping on AUPRC) with tqdm bars
# ---------------------------
best_val_metric = float("-inf")
best_state = None
patience_counter = 0
val_every = int(getattr(config.train, "val_every", 1))
grad_clip = getattr(config.train, "gradient_clip_norm", None)
epochs = int(config.train.epochs)
amp_enabled = rt["amp_enabled"]
amp_dtype   = rt["amp_dtype"]
scaler      = rt["scaler"]

# outer epoch bar
epoch_bar = trange(1, epochs + 1, desc="Epochs", dynamic_ncols=True, leave=True)


for epoch in epoch_bar:
    model.train()
    total_loss = 0.0
    num_nodes = graph_train["encounter"].num_nodes

    # inner batch bar
    batch_bar = tqdm(train_loader,
                     desc=f"Train Epoch {epoch}",
                     dynamic_ncols=True,
                     leave=False)

    avg_loss = train_one_epoch(
        model=model,
        train_loader=train_loader,      # or pass a tqdm wrapper as batch_bar
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        config=config,
        rt=rt,
        batch_bar=batch_bar,            # optional: tqdm(train_loader)
        grad_clip=config.train.gradient_clip_norm
    )

    # step scheduler once per epoch
    if scheduler is not None:
        scheduler.step()

    if "writer" in globals():
        writer.add_scalar("Loss/train", avg_loss, epoch)

    # ---- Validation ----
    val_auprc = None
    if epoch % val_every == 0:
        model.eval()
        with torch.no_grad():
            val_logits = model(val_data.x_dict, val_data.edge_index_dict)
            val_probs = torch.sigmoid(val_logits).detach().cpu().numpy()
            val_labels = graph_val["encounter"].y.detach().cpu().numpy()
            val_auprc = average_precision_score(val_labels, val_probs)

        if "writer" in globals():
            writer.add_scalar("Val/AUPRC", val_auprc, epoch)

        # early stopping
        if val_auprc > best_val_metric:
            best_val_metric = val_auprc
            best_state = {"model": model.state_dict(), "epoch": epoch, "val_auprc": val_auprc}
            patience_counter = 0
        else:
            patience_counter += 1

    # update epoch bar postfix (show both metrics if available)
    if val_auprc is not None:
        epoch_bar.set_postfix(avg_loss=f"{avg_loss:.4f}", val_auprc=f"{val_auprc:.4f}")
        logger.info(f"Epoch {epoch} - Train loss: {avg_loss:.4f}  |  Val AUPRC: {val_auprc:.4f}")
    else:
        epoch_bar.set_postfix(avg_loss=f"{avg_loss:.4f}")
        logger.info(f"Epoch {epoch} - Train loss: {avg_loss:.4f}")

    if patience_counter >= int(config.train.early_stopping_patience):
        tqdm.write("Early stopping triggered.")
        break

# ---------------------------
# Load best weights
# ---------------------------
if best_state:
    model.load_state_dict(best_state["model"])
    logger.info(f"Loaded best model from epoch {best_state['epoch']} with Val AUPRC={best_state['val_auprc']:.4f}")


Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Train Epoch 1:   0%|          | 0/438 [00:01<?, ?it/s]

2025-10-22 17:08:15 INFO:Epoch 1 - Train loss: 0.3917  |  Val AUPRC: 0.1190


Train Epoch 2:   0%|          | 0/438 [00:00<?, ?it/s]

2025-10-22 17:34:07 INFO:Epoch 2 - Train loss: 0.3190  |  Val AUPRC: 0.1201


Train Epoch 3:   0%|          | 0/438 [00:00<?, ?it/s]

In [ ]:
from train.metrics import compute_metrics, find_best_threshold
from train.calibration import calibrate_probabilities, apply_calibration
from train.loader import make_neighbor_loader

# Evaluate on validation set
model.eval()
with torch.no_grad():
    val_out = model(val_data.x_dict, val_data.edge_index_dict)
    test_data = graph_test.to(model.device)
    test_out = model(test_data.x_dict, test_data.edge_index_dict)
val_probs = torch.sigmoid(val_out).cpu().numpy()
test_probs = torch.sigmoid(test_out).cpu().numpy()
val_labels = graph_val['encounter'].y.cpu().numpy()
test_labels = graph_test['encounter'].y.cpu().numpy()

# Compute metrics at default 0.5 threshold
val_metrics = compute_metrics(val_labels, val_probs)
test_metrics = compute_metrics(test_labels, test_probs)
logger.info("Validation metrics at 0.5 threshold: " + ", ".join(f"{k}={v:.4f}" for k,v in val_metrics.items()))
logger.info("Test metrics at 0.5 threshold: " + ", ".join(f"{k}={v:.4f}" for k,v in test_metrics.items()))

# Find best threshold on validation for F1 score
best_thr, best_f1 = find_best_threshold(val_labels, val_probs, optimize_for='f1_pos')
logger.info(f"Best threshold for F1 on val = {best_thr:.2f}, F1 at best thr = {best_f1:.4f}")
# Apply this threshold to test set
test_pred_opt = (test_probs >= best_thr).astype(int)
from sklearn.metrics import f1_score, precision_score, recall_score
f1_test_opt = f1_score(test_labels, test_pred_opt, pos_label=1)
precision_test_opt = precision_score(test_labels, test_pred_opt, pos_label=1, zero_division=0)
recall_test_opt = recall_score(test_labels, test_pred_opt, pos_label=1)
logger.info(f"Test F1={f1_test_opt:.4f}, Precision={precision_test_opt:.4f}, Recall={recall_test_opt:.4f} at threshold {best_thr:.2f}")

# Probability calibration (using validation set)
cal_model = calibrate_probabilities(val_probs, val_labels, method="platt")  # or "isotonic"
cal_test_probs = apply_calibration(cal_model, test_probs)
cal_metrics = compute_metrics(test_labels, cal_test_probs)
logger.info("Test metrics after calibration: " + ", ".join(f"{k}={v:.4f}" for k,v in cal_metrics.items()))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, CalibrationDisplay, ConfusionMatrixDisplay, confusion_matrix

# ROC Curve
RocCurveDisplay.from_predictions(test_labels, test_probs)
plt.title("ROC Curve (Test)")
plt.show()

# Precision-Recall Curve
PrecisionRecallDisplay.from_predictions(test_labels, test_probs)
plt.title("Precision-Recall Curve (Test)")
plt.show()

# Calibration curve (reliability diagram)
CalibrationDisplay.from_predictions(test_labels, test_probs, n_bins=10, strategy='uniform')
plt.title("Calibration Curve (Test)")
plt.show()

# Confusion Matrix at optimal threshold
cm = confusion_matrix(test_labels, test_pred_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Readmit","Readmit<30"])
disp.plot(cmap="Blues")
plt.title(f"Confusion Matrix (Thr={best_thr:.2f})")
plt.show()

# Decision curve analysis (net benefit vs threshold)
from src.train.threshold import decision_curve_analysis
dc_df = decision_curve_analysis(test_labels, test_probs)
plt.plot(dc_df['threshold'], dc_df['net_benefit'], label='GNN Model')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel("Threshold Probability")
plt.ylabel("Net Benefit")
plt.title("Decision Curve (Test Set)")
plt.legend()
plt.show()


In [ ]:
from src.infer.batch_predict import batch_predict

# Suppose we have a CSV "raw/new_encounters.csv" of new patient encounters to predict
new_data_path = "raw/new_encounters.csv"
if os.path.exists(new_data_path):
    predictions_df = batch_predict(new_data_path, model, vocabs, config)
    logger.info(f"Inductive predictions saved to: {config.inference.output_predictions_path}")
    logger.info(predictions_df.head())


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from src.train.metrics import compute_metrics

def train_and_eval_baselines(X_train, y_train, X_val, y_val, X_test, y_test):
    """Train multiple baseline ML models and evaluate on val and test sets."""
    results = {}
    # Define models with some default hyperparameters
    models = {
        "LogisticRegression": LogisticRegression(max_iter=1000),
        "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "LightGBM": LGBMClassifier(random_state=42),
        "SVM": SVC(kernel='rbf', probability=True, random_state=42),
        "KNN": KNeighborsClassifier(n_neighbors=5)
    }
    for name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        # Evaluate on validation set
        if X_val is not None and y_val is not None:
            # Some models (e.g. SVM without probability) we set probability=True for consistency
            val_prob = model.predict_proba(X_val)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_val)
            val_metrics = compute_metrics(y_val, val_prob)
        else:
            val_metrics = {}
        # Evaluate on test set
        test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_test)
        test_metrics = compute_metrics(y_test, test_prob)
        results[name] = {"val_metrics": val_metrics, "test_metrics": test_metrics}
    return results


In [ ]:
from benchmarks.baselines import train_and_eval_baselines

# Prepare NumPy arrays for baseline models
feature_cols = config.data.columns.numeric + config.data.columns.categorical_low_card
X_train_tab = df_train_proc[feature_cols].to_numpy()
y_train_tab = df_train_proc[target_col].to_numpy()
X_val_tab = df_val_proc[feature_cols].to_numpy()
y_val_tab = df_val_proc[target_col].to_numpy()
X_test_tab = df_test_proc[feature_cols].to_numpy()
y_test_tab = df_test_proc[target_col].to_numpy()

# Train and evaluate baseline models
baseline_results = train_and_eval_baselines(X_train_tab, y_train_tab, X_val_tab, y_val_tab, X_test_tab, y_test_tab)

# Display baseline evaluation results
for model_name, metrics_dict in baseline_results.items():
    test_met = metrics_dict["test_metrics"]
    logger.info(f"{model_name} - Test AUROC: {test_met['auroc']:.4f}, AUPRC: {test_met['auprc']:.4f}, "
                f"F1: {test_met['f1_pos']:.4f}, Precision: {test_met['precision_pos']:.4f}, Recall: {test_met['recall_pos']:.4f}, "
                f"Bal Acc: {test_met['balanced_accuracy']:.4f}")


In [17]:
import torch
print("torch:", torch.__version__)      # e.g. 2.2.1
print("cuda :", torch.version.cuda)      # None==CPU, else like '12.1'


torch: 2.2.1
cuda : 12.1
